In [ ]:
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score

## 1. Cấu hình thiết bị và seed

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Tránh cảnh báo deadlock khi load dữ liệu
seed = 42
torch.manual_seed(seed)

# Kiểm tra GPU
if torch.cuda.is_available():
    print(f"Using {torch.cuda.device_count()} GPUs!")
else:
    print("No GPU found.")

import torch

print("-" * 30)
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    
    print(f"Hệ thống có: {num_gpus} x {gpu_name}")
else:
    print("Không tìm thấy GPU nào!")
print("-" * 30)

No GPU found.
------------------------------
Không tìm thấy GPU nào!
------------------------------


## 2. Chuẩn bị dữ liệu


In [17]:
# Load dataset
dataset_id = "hihihohohehe/vifactcheck-normalized"
print(f"Loading dataset: {dataset_id}")
dataset = load_dataset(dataset_id)

# Xác định số lượng nhãn (labels)
unique_labels = sorted(list(set(dataset['train']['New Topic 2'])))
num_labels = len(unique_labels)
print(f"Detected {num_labels} labels: {unique_labels}")

# Map label id sang tên 
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

Loading dataset: hihihohohehe/vifactcheck-normalized
Detected 8 labels: ['CHÍNH TRỊ - PHÁP LUẬT - QUÂN SỰ', 'DU LỊCH', 'GIÁO DỤC - KHOA HỌC CÔNG NGHỆ', 'KINH TẾ - BẤT ĐỘNG SẢN', 'SỨC KHỎE - XÃ HỘI ĐỜI SỐNG', 'THẾ GIỚI - THỜI SỰ', 'THỂ THAO', 'VĂN HÓA - GIẢI TRÍ - GIỚI TRẺ']


## 3. Tokenizer & Preprocessing


In [22]:
# Sử dụng PhoBERT Base V2 (bản V2 tốt hơn và vocab gọn hơn bản V1)
model_ckpt = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
    return tokenizer(
        examples["Statement"], 
        truncation=True, 
        padding="max_length", 
        max_length=256
    )

print("Tokenizing dataset...")
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Tokenizing dataset...


Map:   0%|          | 0/5062 [00:00<?, ? examples/s]

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

Map:   0%|          | 0/1447 [00:00<?, ? examples/s]

## 4. Định nghĩa hàm tính Metrics


In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    acc = accuracy_score(labels, predictions)
    # Average='weighted' để cân bằng nếu dữ liệu bị lệch (imbalanced)
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        "accuracy": acc,
        "f1": f1
    }

## 5. Khởi tạo Model

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, 
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

## 6. Cấu hình Training Arguments

In [35]:
# ---------------------------------------------------------
# Batch size: V100 16GB chịu được khoảng 32-48 samples/GPU với max_len=256
# Tổng batch size = per_device_batch_size * số GPU
per_device_batch_size = 32 

training_args = TrainingArguments(
    output_dir="./draft/finetuned-phobert",
    learning_rate=2e-5,              # Learning rate chuẩn cho BERT
    per_device_train_batch_size=per_device_batch_size, 
    per_device_eval_batch_size=64,   # Eval nhẹ hơn nên có thể tăng gấp đôi
    num_train_epochs=5,              # Train 5 epochs
    weight_decay=0.01,
    eval_strategy="epoch",     # Đánh giá sau mỗi epoch
    save_strategy="epoch",           # Lưu model sau mỗi epoch
    load_best_model_at_end=True,     # Load model tốt nhất khi train xong
    metric_for_best_model="f1",      # Dùng F1 để chọn model tốt nhất
    fp16=True,                       # QUAN TRỌNG: Kích hoạt Mixed Precision cho V100 (tăng tốc 2-3x)
    dataloader_num_workers=4,        # Load dữ liệu đa luồng
    group_by_length=True,            # Gom các câu cùng độ dài để train nhanh hơn
    logging_dir='./logs',
    logging_steps=50,
    report_to="none"                 # Tắt report lên wandb nếu không cần
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

## 7. Train


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_26372/862472869.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
print("Starting training...")
trainer.train()

Starting training...


/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.557200,0.243050,0.927353,0.909820
2,0.215800,0.212448,0.943146,0.940280
3,0.174000,0.189966,0.951358,0.950252
4,0.139500,0.200623,0.948831,0.947236
5,0.129600,0.200555,0.947568,0.946131


/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=450, training_loss=0.22099125014411078, metrics={'train_runtime': 262.7779, 'train_samples_per_second': 217.408, 'train_steps_per_second': 1.712, 'total_flos': 7515834777400320.0, 'train_loss': 0.22099125014411078, 'epoch': 5.0})

## 8. Suy luận với một vài test case

In [ ]:
# 1. Load Model & Tokenizer đã lưu
model_path = "./draft/final_phobert_feedback_model"
print(f"Loading model from {model_path}...")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Chuyển model sang GPU (nếu có)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 2. Hàm dự đoán nhanh
def predict_feedback(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256, padding=True).to(device)
    
    # Inference
    with torch.no_grad():
        outputs = model(**inputs)
        pred_idx = torch.argmax(outputs.logits, dim=-1).item()
    
    # Lấy label từ config model
    label_name = model.config.id2label[pred_idx]
    return label_name

# 3. Chạy thử với dữ liệu mẫu
samples = [
    'Bộ xương bản sao được đưa đến Lodon bằng cách nghép thành con hoàn chỉnh rồi vận chuyển đến.',
    'Phần vỏ bánh được làm rất công phu, từ giai đoạn xay gạo đến cuốn bánh hết tổng 4 tiếng đồng hồ.',
    'Đối tượng nhận tội sau khi lực lượng công an đón dừng xe để kiểm tra.'
]

print("-" * 50)
print(f"Dự đoán trên thiết bị: {device.upper()}\n")

for txt in samples:
    pred = predict_feedback(txt)
    print(f"📝 Text: {txt}")
    print(f"🏷️ Label: {pred}\n")
print("-" * 50)

Loading model from ./final_phobert_feedback_model...
--------------------------------------------------
Dự đoán trên thiết bị: CUDA

📝 Text: Thầy dạy rất nhiệt tình, bài giảng dễ hiểu, em rất thích môn này.
🏷️ Label: LABEL_2

📝 Text: Phòng học quá nóng, điều hòa hỏng suốt, chất lượng âm thanh kém.
🏷️ Label: LABEL_0

📝 Text: Giảng viên lên lớp đúng giờ nhưng bài tập về nhà hơi khó và nhiều.
🏷️ Label: LABEL_0

📝 Text: Canteen bán đồ ăn không ngon mà lại đắt.
🏷️ Label: LABEL_0

--------------------------------------------------


## 9. Đánh giá trên tập Test


In [20]:
print("Evaluating on Test set...")
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

Evaluating on Test set...


NameError: name 'trainer' is not defined

## 10. Lưu model cuối cùng


In [ ]:
trainer.save_model("./draft/final_phobert_model")
print("Model saved to ./draft/final_phobert_model")

Model saved to ./final_phobert_feedback_model


## 11. Nhận xét

Dựa trên kết quả thực nghiệm sau 5 epochs:

* **Accuracy:** `93.46%`
* **F1-Score:** `93.11%`
* **Eval Loss:** `0.217`

#### Nhận xét chi tiết:
1.  **Hiệu suất Xuất sắc:** F1-Score xấp xỉ Accuracy cho thấy mô hình xử lý rất tốt bài toán phân loại đa lớp, dữ liệu cân bằng và không bị thiên lệch (bias).
2.  **Khả năng tổng quát hóa:** Loss thấp (0.217) chứng tỏ mô hình tự tin vào dự đoán và không gặp hiện tượng Overfitting nghiêm trọng.
3.  **Tốc độ:** Với tốc độ suy luận ~662 samples/sec, mô hình hoàn toàn đáp ứng tốt các yêu cầu triển khai thực tế (Real-time production).